In [1]:
import torchio as tio
import SimpleITK as sitk
import os
from tqdm import tqdm
import glob

# 1- Convert DICOM to Nifti

prerequisite: download dcm2niix for the desired OS from here: https://github.com/rordenlab/dcm2niix/releases

In [2]:
def convert_dicom_to_nifti(dicom_dir, output_dir, dcm2niix_path="<path/to>/dcm2niix.exe"):
    """
    Convert DICOM files to Nifti format using dcm2niix.

    Parameters:
        dicom_dir (str): Path to the directory containing DICOM files.
        output_dir (str): Path to the directory where Nifti files will be saved.
        dcm2niix_path (str, optional): Path to the dcm2niix executable. Default is "<path/to>/dcm2niix.exe".

    Returns:
        None
    """
    # Get all the folders in the specified DICOM directory
    dicom_folders = os.listdir(dicom_dir)
    
    # Define the command for dcm2niix conversion
    conversion_command = f"{dcm2niix_path} -z y -f %p_%t_%s -o {output_dir}"
    
    # Iterate through each folder and convert DICOM files to Nifti
    for folder in dicom_folders:
        full_folder_path = os.path.join(dicom_dir, folder)
        os.system(f"{conversion_command} {full_folder_path}")


# 2- Reorient an MRI scan to LPS orientation using SimpleITK. 

In [3]:
def reorient_to_LPS(input_image_path, output_image_path):
    """
    Reorient an MRI scan to LPS orientation using SimpleITK.

    Parameters:
    input_image_path (str): Path to the input MRI image.
    output_image_path (str): Path where the reoriented image will be saved.
    """

    # Read the input image
    input_image = sitk.ReadImage(input_image_path)

    # Reorient the image to LPS orientation
    lps_oriented_image = sitk.DICOMOrient(input_image, 'LPS')

    # Save the reoriented image
    if os.name == "nt":
        out_file_path = os.path.join(output_image_path, input_image_path.split("\\")[-1])
    else:
        out_file_path = os.path.join(output_image_path, input_image_path.split("/")[-1])
    sitk.WriteImage(lps_oriented_image, out_file_path)
    
    
    # verify the orientation
    img = tio.ScalarImage(out_file_path)
    print(img.orientation)

# 3- Register an MRI image to a template using SimpleITK

Rigidly register the scan and resample it to an isotropic resolution (1mm^3) based on common anatomical atlas: SRI.

Link: https://www.nitrc.org/frs/?group_id=214 
file: sri24_spm8.zip

In [4]:
def register_to_mni(input_image_path, template_path, output_path):
    """
    Register an MRI image to the MNI template using SimpleITK.

    Parameters:
    input_image_path (str): Path to the subject's MRI image to be registered.
    template_path (str): Path to the MNI template image.

    Returns:
    SimpleITK.Image: The registered image.
    """

    # Read the subject image and the MNI template
    subject_image = sitk.ReadImage(input_image_path, sitk.sitkFloat32)
    mni_template = sitk.ReadImage(template_path, sitk.sitkFloat32)

    # Initialize the registration method
    registration_method = sitk.ImageRegistrationMethod()

    # Similarity metric settings
    registration_method.SetMetricAsMattesMutualInformation(numberOfHistogramBins=50)
    registration_method.SetMetricSamplingStrategy(registration_method.RANDOM)
    registration_method.SetMetricSamplingPercentage(0.01)

    # Interpolator settings
    registration_method.SetInterpolator(sitk.sitkLinear)

    # Optimizer settings
    registration_method.SetOptimizerAsGradientDescent(learningRate=1.0, numberOfIterations=100, convergenceMinimumValue=1e-6, convergenceWindowSize=10)
    registration_method.SetOptimizerScalesFromPhysicalShift()

    # Setup for the multi-resolution framework
    registration_method.SetShrinkFactorsPerLevel(shrinkFactors=[4, 2, 1])
    registration_method.SetSmoothingSigmasPerLevel(smoothingSigmas=[2, 1, 0])
    registration_method.SmoothingSigmasAreSpecifiedInPhysicalUnitsOn()

    # Don't optimize in-place, we're using a new instance of the image.
    registration_method.SetInitialTransform(sitk.CenteredTransformInitializer(mni_template, subject_image, sitk.Euler3DTransform(), sitk.CenteredTransformInitializerFilter.GEOMETRY))

    # Execute the registration
    final_transform = registration_method.Execute(mni_template, subject_image)

    # Resample the image to the space of the template
    registered_image = sitk.Resample(subject_image, mni_template, final_transform, sitk.sitkLinear, 0.0, subject_image.GetPixelID())
    
    # Save the registered image
    sitk.WriteImage(registered_image, output_path)


In [5]:
"""
Check the orientation and spacing of the registered images
"""

path_file_list = glob.glob(r"<path where files are located>\*.nii.gz")

# check the shape of the images
for path in path_file_list:
    img = tio.ScalarImage(path)
    print(img.orientation)
    print(img.spacing)

# 5- Bias field Correction

In [7]:
def apply_bias_field_correction(parent_dir, path2save):
    """
    Apply bias field correction to images in a directory using N4 bias field correction.

    Parameters:
        parent_dir (str): Path pattern to the directory containing the input images.
        path2save (str): Path to the directory where corrected images will be saved.

    Returns:
        None
    """
    # Find all image files in the parent directory
    parent_file = glob.glob(os.path.join(parent_dir, "*.nii.gz"))
    
    # Apply bias field correction to each image
    for path in tqdm(parent_file):
        # Read the image
        input_image = sitk.ReadImage(path, sitk.sitkFloat32)
        
        # Apply N4 bias field correction
        corrector = sitk.N4BiasFieldCorrectionImageFilter()
        output_image = corrector.Execute(input_image)

        # Define the output file path
        output_file = os.path.join(path2save, os.path.basename(path))
        
        # Save the corrected image
        sitk.WriteImage(output_image, output_file)

# 4- Perform skull stripping using HD-BET from DKFZ
The original repository and instructions are found here: https://github.com/MIC-DKFZ/HD-BET
! Please follow the Installation Instructions in the link

# 5- Bias field Correction

In [7]:
def apply_bias_field_correction(parent_dir, path2save):
    """
    Apply bias field correction to images in a directory using N4 bias field correction.

    Parameters:
        parent_dir (str): Path pattern to the directory containing the input images.
        path2save (str): Path to the directory where corrected images will be saved.

    Returns:
        None
    """
    # Find all image files in the parent directory
    parent_file = glob.glob(os.path.join(parent_dir, "*.nii.gz"))
    
    # Apply bias field correction to each image
    for path in tqdm(parent_file):
        # Read the image
        input_image = sitk.ReadImage(path, sitk.sitkFloat32)
        
        # Apply N4 bias field correction
        corrector = sitk.N4BiasFieldCorrectionImageFilter()
        output_image = corrector.Execute(input_image)

        # Define the output file path
        output_file = os.path.join(path2save, os.path.basename(path))
        
        # Save the corrected image
        sitk.WriteImage(output_image, output_file)

In [6]:
def skull_stripping(parent_dir, out_dir, hd_bet_path="HD-BET/HD_BET/hd-bet"):
    """
    Skull stripping of .nii.gz files using HD-BET tool.

    Parameters:
        parent_dir (str): Path to the directory containing the .nii.gz files.
        out_dir (str): Path to the directory where processed files will be saved.
        hd_bet_path (str): Path to the HD-BET executable. Default is "HD-BET/HD_BET/hd-bet".

    Returns:
        None
    """
    path_file_list = glob.glob(os.path.join(parent_dir, "*.nii.gz"))

    for path in tqdm(path_file_list):
        if os.name == 'nt':  # Check if the operating system is Windows
            out_path = os.path.join(out_file, path.split("\\")[-1])
        else:
            out_path = os.path.join(out_file, path.split("/")[-1])
        os.system(f'python {hd_bet_path} -i {path} -o {out_path}')